# [993. Cousins in Binary Tree](https://leetcode.com/problems/cousins-in-binary-tree/)

In a binary tree, the root node is at depth 0, and children of each depth k node are at depth k+1.

Two nodes of a binary tree are cousins if they have the same depth, but have different parents.

We are given the root of a binary tree with unique values, and the values x and y of two different nodes in the tree.

Return true if and only if the nodes corresponding to the values x and y are cousins.

<font color=red>
总结一下就两条：<br>
    > 节点的深度是否相同<br>
    > 是否为同一个父节点
</font>

## 第一版

入手思路肯定是从遍历每个节点出发，将每个节点的 深度 和 父节点 都存储在一个字典里，即构造一个字典：以当前遍历的节点的值为 key，value 依然为一个字典，分别存储 parent 是谁，以及 depth 是多少。coding 如下：

```
def isCousins(self, root: TreeNode, x: int, y: int) -> bool:
    self.res = {}
    def dfs(node,parent,depth):
        if node:
            left = dfs(node.left,node,depth+1)
            right = dfs(node.right,node,depth+1)
            self.res[node.val] = {'parent':parent,'depth':depth}

    dfs(root,None,0)
    return True if self.res[x]['depth'] == self.res[y]['depth'] and self.res[x]['parent'] != self.res[y]['parent'] else False
```

执行几个测试用例全部正确，submit 也顺利通过，但是效率却有点低。肯定啊，代码是遍历完整个树，而没有中间剪枝的步骤，所以效率肯定是有点低的！

## 第二版

延续第一版的思路，只要找到两个目标节点，便终止遍历，自然可以达到剪枝的目的。

```
class Solution(object):
    def isCousins(self, root, x, y):
        lookup = {}
        def dfs(root, i=0, p=None):
            if root:
                if root.val in (x, y): lookup[root.val] = (i, p)
                dfs(root.left, i=i+1, p=root.val)
                dfs(root.right, i=i+1, p=root.val)
        dfs(root)
        return lookup[x][0] == lookup[y][0] and lookup[x][1] != lookup[y][1]
```

<font color=red>
更好的做法：
</font>

```
def isCousins(self, root: TreeNode, x: int, y: int) -> bool:
    def dfs(node,parent,depth,target):
        if node:
            if node.val == target:
                return parent,depth
            return dfs(node.left,node,depth+1,target) or dfs(node.right,node,depth+1,target)

    parent_x, depth_x = dfs(root,None,0,x)
    parent_y, depth_y = dfs(root,None,0,y)
    return depth_x == depth_y and parent_x != parent_y
```

事实证明，效率确实提高了很多！未借助额外的字典结构，耗时也大幅缩小了！

## 迭代版

BFS 方法，按层寻找，应该是最有效的方法了！

```
    def isCousins(self, root: TreeNode, x: int, y: int) -> bool:
        queue = [root]
        
        # 按层遍历开始
        while len(queue) > 0:
            size = len(queue)
            is_x_exist = False
            is_y_exist = False
            
            # 将当前队列中的所有节点向四周扩散
            for i in range(size):
                current = queue.pop(0)
                if current.val == x:
                    is_x_exist = True
                    
                if current.val == y:
                    is_y_exist = True
                
                # 如果在当前层找到 x 和 y,但是因为二者是一个父节点，故返回 False!
                if current.left and current.right:
                    if current.left.val == x and current.right.val == y:
                        return False
                    
                    if current.right.val == x and current.left.val == y:
                        return False
                    
                if current.left:
                    queue.append(current.left)
                    
                if current.right:
                    queue.append(current.right)
            # 遍历完当前层，仍然找到 x 和 y,并且二者不是一个父节点，则可以返回 True.
            if is_x_exist and is_y_exist:
                return True
            
        return False
```